In [1]:
import numpy as np

In [41]:
n = 100

def foo(i, j, k):
    return np.sin((i + j + k)) + np.cos(3 * (i + j + k))

A = np.fromfunction(foo, [n, n, n])
d = len(A.shape)

In [42]:
def crop_vector(a, eps):
    r = 0
    cur_sum = 0
    sum = np.sum(a)
    while sum - cur_sum >= eps:
        cur_sum += a[r]
        r += 1
    return r + 1

def tensor_unfold(A, k):
    return np.reshape(np.moveaxis(A, k, 0), (A.shape[k], -1))

def mul_by_dim(T, A, k):
    d = len(T.shape)
    return np.einsum(T, [i for i in range(d)], A, [k, d], [i if i != k else d for i in range(d)])

In [43]:
def stHOSVD(A, eps=1e-5):
    d = len(A.shape)
    U = []
    G = A
    shape = [k for k in A.shape]
    for k in range(d):
        Ak = tensor_unfold(G, k)
        u, s, v = np.linalg.svd(Ak, full_matrices=False)
        r = crop_vector(s, eps / d**0.5)
        shape[k] = r
        U.append(u[:, :r])
        G = np.diag(s[:r]) @ v[:r, :]
        G = G.reshape(r, *[shape[i] for i in range(len(shape)) if i != k])
    return np.transpose(G), U  # ranks are inverse in G  


In [44]:
g, U = stHOSVD(A)
g.shape

(5, 5, 5)

In [45]:
T = g
for k in range(d):
    T = mul_by_dim(T, U[k].T, k)

np.linalg.norm(T - A)

9.954122877747075e-12